In [1]:
# Import libraries and packages for the project 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import random


print('- Finish importing packages')

- Finish importing packages


In [2]:
shopee_url = 'https://shopee.vn'

In [3]:
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
# wait = WebDriverWait(driver,30)
url = 'https://shopee.vn/mall/brands/'
sleep(2)
driver.get(url)
print('- Finish initializing a driver')


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [/home/dijkstra/.wdm/drivers/chromedriver/linux64/104.0.5112.20/chromedriver] found in cache


- Finish initializing a driver


In [4]:
page_source = BeautifulSoup(driver.page_source)
sections = page_source.find_all('div', {"class":["official-shop-brand-list__section-content"]})
order = 1
count = 0
shop_list = []
headers = ['Order', 'Name', 'URL']

def handle_add_shop(order,name,shop_URL):
    shop_list.append({headers[0]:order,headers[1]:name,headers[2]:shop_URL})
    
    print("Added new shop: ")
    print("%d - %s - %s"%(order,name,shop_URL))


for section in sections:
    order_section = section.findAll('div',class_="full-brand-list-item")
    if(order==101): # Get enough data
        break
    for shop in order_section:
        cmt_flag = False

        if(order==101):
            break
        # Random order shop
        is_next = random.randint(1,10)
        if(is_next%2==0):
            continue

        shop_URL = shopee_url + shop.find('a', class_="full-brand-list-item__brand-cover-image").get('href') # Get shop url
        name = shop.find('div',class_="full-brand-list-item__brand-name").text # Get shop name
        try:
            driver.get(shop_URL)
            print('- Accessing shop: ', shop_URL)
            sleep(2)
            driver.execute_script('window.scrollTo(0, 0);') #scroll to the end of the page
            sleep(1)
        except TimeoutException:
            print("***************************")
            print("Loading took too much time!")
            print("***************************")
            continue


        loop_page_source = BeautifulSoup(driver.page_source,"html.parser")
        items = loop_page_source.find_all('div',{"class":["section-seller-overview__item-text-value"]})
        if(items):
            if(items[0] and items[8]):
                product_quantity = items[0]
                comment_quantity = items[8]
                # Check product quantity
                product_quantity = product_quantity.text 

                # Check comment
                comment_items = comment_quantity.text.split(' ')
                if(len(comment_items)!=4):
                    continue
                comment_quantity = comment_items[1][1:]
                if(not comment_quantity.isdecimal()):
                    tmp = comment_quantity.split(",")

                    cmt_quantity = tmp[0] if len(tmp)>1 else tmp[0][:-1]
                    if(int(cmt_quantity)>=20):
                        cmt_flag = True
                    
                if(cmt_flag):
                    if(not product_quantity.isdecimal()): # Quantity > 100
                        handle_add_shop(order,name,shop_URL)
                        count+=1
                        order+=1  
                    elif(int(product_quantity)>=100): # Check shop product quantity is greater than 100 or not
                        handle_add_shop(order,name,shop_URL)
                        count+=1
                        order+=1    

                sleep(3)

                if(count==10): # 
                    count = 0
                    break
            else:
                continue

        
print("Finished crawling data!")
        

- Accessing shop:  https://shopee.vn/a_chau_books
- Accessing shop:  https://shopee.vn/atous.clothing
- Accessing shop:  https://shopee.vn/aaajeans
- Accessing shop:  https://shopee.vn/antahn_official
- Accessing shop:  https://shopee.vn/abinbevvietnam
- Accessing shop:  https://shopee.vn/abbie.oh
- Accessing shop:  https://shopee.vn/abbottofficial
Added new shop: 
1 - Abbott - https://shopee.vn/abbottofficial
- Accessing shop:  https://shopee.vn/abbottfreestylelibre
- Accessing shop:  https://shopee.vn/abbyschoice.vn
- Accessing shop:  https://shopee.vn/baongochb93
- Accessing shop:  https://shopee.vn/abc123.vn
- Accessing shop:  https://shopee.vn/abcare.vn
- Accessing shop:  https://shopee.vn/abg_official_store
- Accessing shop:  https://shopee.vn/absolutenewyork.official
- Accessing shop:  https://shopee.vn/acmenfashion
- Accessing shop:  https://shopee.vn/academyvietstem
- Accessing shop:  https://shopee.vn/acehomecentervn
- Accessing shop:  https://shopee.vn/acer_flagshipstore
- A

In [5]:
with open('ShopeeMall.csv', 'w',  newline = '') as file_output:
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for shop in shop_list:
        writer.writerow(shop)

print('Mission Completed!')

Mission Completed!
